In [1]:
import tensorflow as tf
import os
from tensorflow.keras.layers import *
from tensorflow.keras.models import Model
from tensorflow.keras import backend as K
AUTOTUNE = tf.data.experimental.AUTOTUNE
import multiprocessing
import numpy as np
from IPython.display import clear_output

2023-10-01 12:34:32.517645: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-10-01 12:34:33.136722: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [5]:
from tensorflow.keras.utils import custom_object_scope
from tensorflow.keras.models import load_model

In [6]:
LR = 0.001
# Custom loss function
def bce_dice_loss(y_true, y_pred):
    y_true = tf.cast(y_true, dtype=tf.float32)  # Привести y_true к типу float32
    # dice_loss = dice_coef(y_true, y_pred)
    bce_loss = tf.keras.losses.binary_crossentropy(y_true, y_pred)
    return bce_loss

def dice_coef(y_true, y_pred, smooth=1e-5):
    y_true_f = tf.keras.layers.Flatten()(y_true)
    y_pred_f = tf.keras.layers.Flatten()(y_pred)
    intersection = tf.reduce_sum(y_true_f * y_pred_f)
    union = tf.reduce_sum(y_true_f) + tf.reduce_sum(y_pred_f)
    return (2.0 * intersection + smooth) / (union + smooth)

NUM_EPOCHS=25
IMG_WIDTH       = 512
IMG_HEIGHT      = 512
IMG_CHANNELS    = 3

In [7]:
@tf.function
def load_image_and_mask(image_path):
    image = tf.io.read_file(image_path)
    image = tf.image.decode_jpeg(image, channels=3)
    image = tf.image.resize(image, (IMG_HEIGHT, IMG_WIDTH), method="bicubic")
    image = tf.cast(image, dtype=tf.float32) / 255.
    return image

In [8]:
# with custom_object_scope({'bce_dice_loss': bce_dice_loss}):
model = load_model('./mod')
for lay in model.layers:
    lay.trainable = False

2023-10-01 12:35:06.158428: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:995] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-10-01 12:35:06.176381: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:995] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-10-01 12:35:06.176613: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:995] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysf

In [9]:
image = tf.io.read_file("/home/denis/code/gas-spread-CV/data_for_gan/input/1.jpg")
image = tf.image.decode_jpeg(image, channels=3)
image = tf.image.resize(image, (IMG_HEIGHT, IMG_WIDTH), method="bicubic")
image = tf.cast(image, dtype=tf.float32) / 255.
pred = model.predict(tf.expand_dims(image, 0))

2023-10-01 12:35:13.020617: I tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:432] Loaded cuDNN version 8600
2023-10-01 12:35:13.693757: W tensorflow/tsl/framework/bfc_allocator.cc:296] Allocator (GPU_0_bfc) ran out of memory trying to allocate 4.08GiB with freed_by_count=0. The caller indicates that this is not a failure, but this may mean that there could be performance gains if more memory were available.
2023-10-01 12:35:13.909041: W tensorflow/tsl/framework/bfc_allocator.cc:296] Allocator (GPU_0_bfc) ran out of memory trying to allocate 4.09GiB with freed_by_count=0. The caller indicates that this is not a failure, but this may mean that there could be performance gains if more memory were available.
2023-10-01 12:35:14.431081: W tensorflow/tsl/framework/bfc_allocator.cc:296] Allocator (GPU_0_bfc) ran out of memory trying to allocate 4.15GiB with freed_by_count=0. The caller indicates that this is not a failure, but this may mean that there could be performance gains if m

1/1 [==============================] - 5s 5s/step


In [ ]:
np.argmax(pred)